In [1]:
import torch
from torch import nn
import torchvision
from torchvision import transforms

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [4]:
train_dataset = torchvision.datasets.MNIST(root='data',
                                          train=True,
                                          download=True,
                                          transform=transforms.ToTensor())

test_dataset = torchvision.datasets.MNIST(root='data',
                                         train=False,
                                         download=True,
                                         transform=transforms.ToTensor())

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2  = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [7]:
model = NeuralNetwork(input_size, hidden_size, num_classes).to(device)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (img, label) in enumerate(train_loader):
        img = img.reshape(-1, 28*28).to(device)
        label = label.to(device)
        
        output = model(img)
        loss = criterion(output, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if(i+1) % 100 == 0:
            print('Epoch: [{}/{}], Step: [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch: [1/5], Step: [100/600], Loss: 0.2914
Epoch: [1/5], Step: [200/600], Loss: 0.5347
Epoch: [1/5], Step: [300/600], Loss: 0.2182
Epoch: [1/5], Step: [400/600], Loss: 0.2160
Epoch: [1/5], Step: [500/600], Loss: 0.2449
Epoch: [1/5], Step: [600/600], Loss: 0.1043
Epoch: [2/5], Step: [100/600], Loss: 0.1123
Epoch: [2/5], Step: [200/600], Loss: 0.0908
Epoch: [2/5], Step: [300/600], Loss: 0.1022
Epoch: [2/5], Step: [400/600], Loss: 0.1521
Epoch: [2/5], Step: [500/600], Loss: 0.0757
Epoch: [2/5], Step: [600/600], Loss: 0.1053
Epoch: [3/5], Step: [100/600], Loss: 0.0679
Epoch: [3/5], Step: [200/600], Loss: 0.0746
Epoch: [3/5], Step: [300/600], Loss: 0.1763
Epoch: [3/5], Step: [400/600], Loss: 0.1015
Epoch: [3/5], Step: [500/600], Loss: 0.0688
Epoch: [3/5], Step: [600/600], Loss: 0.0307
Epoch: [4/5], Step: [100/600], Loss: 0.0197
Epoch: [4/5], Step: [200/600], Loss: 0.0375
Epoch: [4/5], Step: [300/600], Loss: 0.0754
Epoch: [4/5], Step: [400/600], Loss: 0.0495
Epoch: [4/5], Step: [500/600], L

In [10]:
with torch.no_grad():
    correct = 0
    total = 0
    for img, label in test_loader:
        img = img.reshape(-1, 28*28).to(device)
        label = label.to(device)
        output = model(img)
        _, predicted = torch.max(output.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()
    
    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 98.01 %
